# Fake News Classifier using NLP and Deep Leanring

Dataset from kaggle: https://www.kaggle.com/c/fake-news/data

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
# Drop NA values
df.dropna(inplace=True)

In [5]:
# Getting independent features

X = df.drop(['label'],axis=1)
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [6]:
# Get dependent feature
y = df['label']

In [7]:
X.shape, y.shape

((18285, 4), (18285,))

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
# Setting up vocab size
voc_size = 5000 # assuming text of any article is less then 5k

## One hot representation 

Finding indexes of word in vocabulary for each sentence


In [13]:
messages = X.copy()

In [14]:
messages.reset_index(drop=True,inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
### Dataset Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
# one hot representation 
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr[:10]

In [23]:
max_sent_length = [len(word) for word in onehot_repr]
print(max(max_sent_length))

12059


## Embedding representation

Padding the sequences since each sentence in the article is of different length

In [24]:
sent_length = 12060

embedded_docs = pad_sequences(onehot_repr,padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  606 1343 2240]
 [   0    0    0 ... 4399  283 1701]
 [   0    0    0 ... 1838 4073 3538]
 ...
 [   0    0    0 ... 1308 3359 2574]
 [   0    0    0 ... 3828 3010 2363]
 [   0    0    0 ...  603 1467  881]]


In [26]:
embedded_docs[0]

array([   0,    0,    0, ...,  606, 1343, 2240], dtype=int32)

In [25]:
len(embedded_docs) # to be trained on model

18285

In [27]:
# Creating model
embedding_model_features = 100

model = Sequential()
model.add(Embedding(voc_size, embedding_model_features, input_length=sent_length))
model.add(LSTM(200))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 12060, 100)        500000    
_________________________________________________________________
lstm (LSTM)                  (None, 200)               240800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 741,001
Trainable params: 741,001
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Splitting the dataset

import numpy as np

X_final = np.array(embedded_docs)
y_final = np.array(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [30]:
X_train.shape, X_test.shape, y_train.shape

((12250, 12060), (6035, 12060), (12250,))

## Training the model

In [31]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, verbose=1, batch_size=64)

Epoch 1/10
192/192 [==============================] - 210s 1s/step - loss: 0.3896 - accuracy: 0.8064 - val_loss: 0.3822 - val_accuracy: 0.8212
Epoch 2/10
192/192 [==============================] - 203s 1s/step - loss: 0.2018 - accuracy: 0.9214 - val_loss: 0.2989 - val_accuracy: 0.8830
Epoch 3/10
192/192 [==============================] - 203s 1s/step - loss: 0.1420 - accuracy: 0.9487 - val_loss: 0.2480 - val_accuracy: 0.9056
Epoch 4/10
192/192 [==============================] - 202s 1s/step - loss: 0.1309 - accuracy: 0.9513 - val_loss: 0.3232 - val_accuracy: 0.8880
Epoch 5/10
192/192 [==============================] - 203s 1s/step - loss: 0.0776 - accuracy: 0.9731 - val_loss: 0.2895 - val_accuracy: 0.9032
Epoch 6/10
192/192 [==============================] - 203s 1s/step - loss: 0.0990 - accuracy: 0.9612 - val_loss: 0.2929 - val_accuracy: 0.9170
Epoch 7/10
192/192 [==============================] - 203s 1s/step - loss: 0.0407 - accuracy: 0.9869 - val_loss: 0.3420 - val_accuracy: 0.9130

In [34]:
model.save('/content/drive/MyDrive/fake_news_lstm_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/fake_news_lstm_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/fake_news_lstm_model/assets


In [35]:
# predicting on test set
from sklearn import metrics

pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [37]:
cm = metrics.confusion_matrix(y_test, pred)
print(cm)

[[3142  277]
 [ 304 2312]]
